# Frictionless Product Categorization

- **Author:** Jose Vicente Ruiz Cepeda (jr3660)
- **Course:** COMS W 4995 - Deep Learning for Computer Vision
- **Assignment**: Final project

## Context

Explain some of the context of the problem here.

## Code

### Imports

In [1]:
import keras # Keras 1.2.2 assumed.
from keras import optimizers

from keras.models import Model

from keras.applications import ResNet50
from keras.applications import InceptionV3
from keras.applications import Xception
from keras.applications import VGG16
from keras.applications import VGG19

from keras.applications.imagenet_utils import preprocess_input
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.inception_v3 import preprocess_input

from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.preprocessing.image import ImageDataGenerator

from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import ZeroPadding2D
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import Flatten

from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Required to avoid errors with the images.
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

%matplotlib inline

Using TensorFlow backend.


### Constants

In [2]:
CLASS_NAMES = {
    0: 'photo',
    1: 'electronics',
    2: 'events',
    3: 'instruments',
    4: 'tools',
    5: 'sports',
    6: 'caravans',
    7: 'others'
}

### Part I - Categories

In [3]:
train_data_dir = './data/train'
validation_data_dir = './data/validation'
test_data_dir = './data/test'

In [4]:
class Categorizer:
    def __init__(self, num_classes, num_dense_layer_units=256, dropout=0.5):
        self.num_classes = num_classes
        self.num_dense_layer_units = num_dense_layer_units
        self.dropout = dropout
        
    def build_model(self, ImagenetModel=ResNet50, verbose=False):
        # Keras models are functions, not classes, so we have to check which one is it like this.
        if ImagenetModel.func_name == 'InceptionV3' or ImagenetModel.func_name == 'Xception':
            self.img_width, self.img_height = 299, 299
        else:
            self.img_width, self.img_height = 224, 224

        # First, let's load the model with ImageNet weights and without the top layer.
        # This will take some time the first time, since the weights have to be downloaded.
        model = ImagenetModel(
            weights='imagenet',
            include_top=False,
            input_shape=(self.img_width, self.img_height, 3)
        )

        # Now, let's create the top layers adapted to our problem. For that we use the
        # Functional API of Keras. (https://keras.io/getting-started/functional-api-guide/)
        x = model.output
        x = Flatten(input_shape=model.output_shape[1:])(x)
        x = Dense(self.num_dense_layer_units, activation='relu')(x)
        x = Dropout(self.dropout)(x)
        preds = Dense(self.num_classes, activation='softmax')(x)
        self.num_top_layers = 4

        # Combine both models to get the final one.
        self.model = Model(model.input, preds)

        if verbose:
            self.model.summary()

        return self
    
    def compile_model(self, optimizer=optimizers.Adagrad(lr=0.001)):
        self.is_compiled = True
        self.optimizer = optimizer
        self.model.compile(
            loss='categorical_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy']
        )
            
        return self
    
    def _recompile_model(self):
        if self.is_compiled != True:
            raise Error("Model has to be compiled first.")

        self.model.compile(
            loss='categorical_crossentropy',
            optimizer=self.optimizer,
            metrics=['accuracy']
        )
    
    def fine_tune(self,
                  class_names,
                  train_data_dir,
                  validation_data_dir,
                  batch_size=16,
                  num_only_top_epochs=10,
                  num_whole_model_epochs=40,
                  best_model_path=None,
                  tensorboard_logs_path=None,
                  reduce_learning_rate=True):
        
        # Augmentation configurations for training and validation.
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True)
        validation_datagen = ImageDataGenerator(rescale=1./255)

        train_generator = train_datagen.flow_from_directory(
            train_data_dir,
            target_size=(self.img_width, self.img_height),
            batch_size=batch_size,
            classes=class_names,
            class_mode='categorical')

        validation_generator = validation_datagen.flow_from_directory(
            validation_data_dir,
            target_size=(self.img_width, self.img_height),
            batch_size=batch_size,
            classes=class_names,
            class_mode='categorical')
        
        if num_only_top_epochs > 0:
            # Freeze everything except the top layers, before training.
            for layer in self.model.layers[:-self.num_top_layers]:
                layer.trainable = False 

            print "Starting with top layers training..."
            self._fit_generator(
                train_generator,
                validation_generator,
                batch_size,
                num_only_top_epochs,
                best_model_path=best_model_path,
                tensorboard_logs_path=tensorboard_logs_path,
                tensorboard_logs_path_suffix='Top',
                reduce_learning_rate=True)
            print "Top layers training done."
            
        if num_whole_model_epochs > 0:
            # Unfreeze everything and train for some more epochs.
            for layer in self.model.layers:
                layer.trainable = True
            
            print "Starting with whole model training..."
            self._fit_generator(
                train_generator,
                validation_generator,
                batch_size,
                num_whole_model_epochs,
                best_model_path=best_model_path,
                tensorboard_logs_path=tensorboard_logs_path,
                tensorboard_logs_path_suffix='Whole',
                reduce_learning_rate=True)
            print "Whole model training done"
        
        return self
    
    def _fit_generator(self,
                       train_generator,
                       validation_generator,
                       batch_size,
                       num_epochs,
                       best_model_path=None,
                       tensorboard_logs_path=None,
                       tensorboard_logs_path_suffix='',
                       reduce_learning_rate=True):
        # Recompile model before training to increase efficiency in
        # case of frozen layers.
        self._recompile_model()
        
        # Save the best model based on validation accuracy.
        if best_model_path:
            model_checkpoint = ModelCheckpoint(
                best_model_path,
                monitor='val_acc',
                save_best_only=True,
                save_weights_only=False,
                period=1
            )
        
        # Log epochs information like loss and accuracy to review it
        # afterwards using TensorBoard.
        if tensorboard_logs_path:
            if tensorboard_logs_path[-1] == '/':
                tensorboard_logs_path = tensorboard_logs_path[:-1]
            tensorboard = TensorBoard(
                log_dir=tensorboard_logs_path+'-'+tensorboard_logs_path_suffix+'/')
        
        # Reduce learning rate i
        if reduce_learning_rate:
            reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001)
        
        self.model.fit_generator(
            train_generator,
            samples_per_epoch=train_generator.n//batch_size,
            nb_epoch=num_epochs,
            validation_data=validation_generator,
            nb_val_samples=validation_generator.n//batch_size,
            callbacks=[model_checkpoint, tensorboard])

In [5]:
print "Training VGG16"
categorizer = Categorizer(len(CLASS_NAMES)). \
    build_model(VGG16). \
    compile_model(). \
    fine_tune(
        CLASS_NAMES.values(),
        train_data_dir,
        validation_data_dir,
        batch_size=16,
        num_only_top_epochs=20,
        num_whole_model_epochs=80,
        best_model_path='best_models/VGG16.hdf5',
        tensorboard_logs_path='tensorboard_logs/VGG16'
    )
del categorizer

Training VGG16
Found 13929 images belonging to 8 classes.
Found 3489 images belonging to 8 classes.
Starting with top layers training...
Epoch 1/20
864/870 [============================>.] - ETA: 0s - loss: 2.3606 - acc: 0.2998

/usr/local/lib/python2.7/dist-packages/keras/engine/training.py:1573: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


880/870 [==============================] - 22s - loss: 2.3455 - acc: 0.3034 - val_loss: 1.4900 - val_acc: 0.4911
Epoch 2/20
880/870 [==============================] - 20s - loss: 1.6454 - acc: 0.3761 - val_loss: 1.4730 - val_acc: 0.4643
Epoch 3/20
880/870 [==============================] - 20s - loss: 1.5873 - acc: 0.4136 - val_loss: 1.5075 - val_acc: 0.4464
Epoch 4/20
864/870 [============================>.] - ETA: 0s - loss: 1.5814 - acc: 0.3970

/usr/lib/python2.7/dist-packages/PIL/Image.py:857: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


880/870 [==============================] - 21s - loss: 1.5808 - acc: 0.3955 - val_loss: 1.4112 - val_acc: 0.5089
Epoch 5/20
880/870 [==============================] - 20s - loss: 1.5205 - acc: 0.4557 - val_loss: 1.3608 - val_acc: 0.4955
Epoch 6/20
880/870 [==============================] - 20s - loss: 1.4816 - acc: 0.4557 - val_loss: 1.4304 - val_acc: 0.4777
Epoch 7/20
880/870 [==============================] - 21s - loss: 1.5073 - acc: 0.4432 - val_loss: 1.3421 - val_acc: 0.5536
Epoch 8/20
880/870 [==============================] - 20s - loss: 1.5197 - acc: 0.4398 - val_loss: 1.3651 - val_acc: 0.4821
Epoch 9/20
880/870 [==============================] - 20s - loss: 1.5023 - acc: 0.4580 - val_loss: 1.3989 - val_acc: 0.4732
Epoch 10/20
880/870 [==============================] - 21s - loss: 1.4493 - acc: 0.4625 - val_loss: 1.3271 - val_acc: 0.5022
Epoch 11/20
880/870 [==============================] - 20s - loss: 1.3771 - acc: 0.4864 - val_loss: 1.3243 - val_acc: 0.5179
Epoch 12/20
880/8

In [6]:
print "Training VGG19"
categorizer = Categorizer(len(CLASS_NAMES)). \
    build_model(VGG19). \
    compile_model(). \
    fine_tune(
        CLASS_NAMES.values(),
        train_data_dir,
        validation_data_dir,
        batch_size=16,
        num_only_top_epochs=20,
        num_whole_model_epochs=80,
        best_model_path='best_models/VGG19.hdf5',
        tensorboard_logs_path='tensorboard_logs/VGG19'
    )
del categorizer

Training VGG19
Found 13929 images belonging to 8 classes.
Found 3489 images belonging to 8 classes.
Starting with top layers training...
Epoch 1/20
880/870 [==============================] - 26s - loss: 2.4984 - acc: 0.3080 - val_loss: 1.6231 - val_acc: 0.3884
Epoch 2/20
880/870 [==============================] - 26s - loss: 1.7209 - acc: 0.3648 - val_loss: 1.5196 - val_acc: 0.4598
Epoch 3/20
880/870 [==============================] - 26s - loss: 1.7002 - acc: 0.3523 - val_loss: 1.5050 - val_acc: 0.4643
Epoch 4/20
880/870 [==============================] - 25s - loss: 1.5933 - acc: 0.3875 - val_loss: 1.4419 - val_acc: 0.4018
Epoch 5/20
880/870 [==============================] - 26s - loss: 1.6020 - acc: 0.4216 - val_loss: 1.4453 - val_acc: 0.4821
Epoch 6/20
880/870 [==============================] - 25s - loss: 1.5567 - acc: 0.4318 - val_loss: 1.4306 - val_acc: 0.4688
Epoch 7/20
880/870 [==============================] - 26s - loss: 1.5280 - acc: 0.4455 - val_loss: 1.3448 - val_acc: 0.

In [7]:
print "Training Xception"
categorizer = Categorizer(len(CLASS_NAMES)). \
    build_model(Xception). \
    compile_model(). \
    fine_tune(
        CLASS_NAMES.values(),
        train_data_dir,
        validation_data_dir,
        batch_size=16,
        num_only_top_epochs=20,
        num_whole_model_epochs=80,
        best_model_path='best_models/Xception.hdf5',
        tensorboard_logs_path='tensorboard_logs/Xception'
    )
del categorizer

Training Xception
Found 13929 images belonging to 8 classes.
Found 3489 images belonging to 8 classes.
Starting with top layers training...
Epoch 1/20
880/870 [==============================] - 37s - loss: 10.3835 - acc: 0.3057 - val_loss: 10.1091 - val_acc: 0.3571
Epoch 2/20
880/870 [==============================] - 38s - loss: 9.1187 - acc: 0.3955 - val_loss: 8.2808 - val_acc: 0.4241
Epoch 3/20
880/870 [==============================] - 38s - loss: 8.6435 - acc: 0.4057 - val_loss: 7.5051 - val_acc: 0.5000
Epoch 4/20
880/870 [==============================] - 34s - loss: 8.7242 - acc: 0.4102 - val_loss: 7.3403 - val_acc: 0.4688
Epoch 5/20
880/870 [==============================] - 34s - loss: 7.7521 - acc: 0.4523 - val_loss: 7.3592 - val_acc: 0.4554
Epoch 6/20
880/870 [==============================] - 35s - loss: 7.7618 - acc: 0.4455 - val_loss: 7.5866 - val_acc: 0.4509
Epoch 7/20
880/870 [==============================] - 40s - loss: 6.9938 - acc: 0.4534 - val_loss: 4.9083 - val_ac

In [8]:
print "Training InceptionV3"
categorizer = Categorizer(len(CLASS_NAMES)). \
    build_model(InceptionV3). \
    compile_model(). \
    fine_tune(
        CLASS_NAMES.values(),
        train_data_dir,
        validation_data_dir,
        batch_size=16,
        num_only_top_epochs=20,
        num_whole_model_epochs=80,
        best_model_path='best_models/InceptionV3.hdf5',
        tensorboard_logs_path='tensorboard_logs/InceptionV3'
    )
del categorizer

Training InceptionV3
Found 13929 images belonging to 8 classes.
Found 3489 images belonging to 8 classes.
Starting with top layers training...
Epoch 1/20
880/870 [==============================] - 65s - loss: 9.0356 - acc: 0.3511 - val_loss: 8.7741 - val_acc: 0.3839
Epoch 2/20
880/870 [==============================] - 26s - loss: 8.0852 - acc: 0.4227 - val_loss: 7.9654 - val_acc: 0.4375
Epoch 3/20
880/870 [==============================] - 26s - loss: 7.6998 - acc: 0.4534 - val_loss: 6.9515 - val_acc: 0.5045
Epoch 4/20
880/870 [==============================] - 26s - loss: 7.1624 - acc: 0.4830 - val_loss: 6.4973 - val_acc: 0.5312
Epoch 5/20
880/870 [==============================] - 26s - loss: 7.0213 - acc: 0.4966 - val_loss: 6.2507 - val_acc: 0.5491
Epoch 6/20
880/870 [==============================] - 22s - loss: 7.3577 - acc: 0.4761 - val_loss: 6.9894 - val_acc: 0.4911
Epoch 7/20
880/870 [==============================] - 26s - loss: 7.0274 - acc: 0.4761 - val_loss: 5.2872 - val_a

In [9]:
print "Training ResNet50"
categorizer = Categorizer(len(CLASS_NAMES)). \
    build_model(ResNet50). \
    compile_model(). \
    fine_tune(
        CLASS_NAMES.values(),
        train_data_dir,
        validation_data_dir,
        batch_size=16,
        num_only_top_epochs=20,
        num_whole_model_epochs=80,
        best_model_path='best_models/ResNet50.hdf5',
        tensorboard_logs_path='tensorboard_logs/ResNet50'
    )
del categorizer

Training ResNet50
Found 13929 images belonging to 8 classes.
Found 3489 images belonging to 8 classes.
Starting with top layers training...
Epoch 1/20
880/870 [==============================] - 46s - loss: 1.9810 - acc: 0.3159 - val_loss: 1.9391 - val_acc: 0.1920
Epoch 2/20
880/870 [==============================] - 14s - loss: 1.6318 - acc: 0.4170 - val_loss: 1.9563 - val_acc: 0.1116
Epoch 3/20
880/870 [==============================] - 15s - loss: 1.5375 - acc: 0.4523 - val_loss: 1.9425 - val_acc: 0.2366
Epoch 4/20
880/870 [==============================] - 15s - loss: 1.5408 - acc: 0.4443 - val_loss: 1.9916 - val_acc: 0.2455
Epoch 5/20
880/870 [==============================] - 15s - loss: 1.4721 - acc: 0.4625 - val_loss: 1.9145 - val_acc: 0.3170
Epoch 6/20
880/870 [==============================] - 14s - loss: 1.4432 - acc: 0.4727 - val_loss: 1.9072 - val_acc: 0.2679
Epoch 7/20
880/870 [==============================] - 15s - loss: 1.3641 - acc: 0.5170 - val_loss: 1.8334 - val_acc:

# Sandbox

img_path = '/Users/Josevi/product_images/photo/9760-1.jpg'
img = load_img(img_path, target_size=(224, 224))
x = img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

imgplot = plt.imshow(mpimg.imread(img_path))
plt.show()

%time
preds = model.predict(x)
print preds
#print('Predicted:', decode_predictions(preds))

def show_sample(X, y, prediction=-1):
    im = X
    print y
    #y = np.flip(y, axis=0)
    y_label = class_name[np.nonzero(y)[0][0]]
    plt.imshow(im)
    if prediction >= 0:
        plt.title("Class = %s, Predict = %s" % (y_label, class_name[prediction]))
    else:
        plt.title("Class = %s" % (y_label))

    plt.axis('on')
    plt.show()

for X_batch, Y_batch in train_generator:
    for i in range(len(Y_batch)):
        show_sample(X_batch[i, :, :, :], Y_batch[i])
    break